## Plan transfers between areas

This notebook shows how we can use Decision Optimization to optimize the move of affected people between hospitals to avoid being overcapacity.

The problem is solved per department.

There are 5 parts in this notebooks:
1. Load the data form different places (departements, current situation, etc)
2. represent the current situation on a map
3. predict new cases to come for each department
4. plan transfers 
5. display all the transfers from the solution

**DISCLAIMER: this notebook is thought to demonstrate through an example how Machine Learning and Decision Optmization could be used, but it is partially based on fake data and knowledge of the real problem, and some significant work would be required to incorporate real data and real constraints and obejctives so that the outcome could be useful.**

In [ ]:
pip install brunel

In [ ]:
import brunel
import pandas as pd
import math
import csv
import collections

### 1. Load the data

Let's import data on hospital status (from the `data.gouf.fr` site).

See https://www.data.gouv.fr/fr/datasets/donnees-relatives-a-lepidemie-de-covid-19/#_

In [ ]:
url ="https://www.data.gouv.fr/fr/datasets/r/63352e38-d353-4b54-bfd1-f1b3ee1cabd7"
df=pd.read_csv(url, sep=";")
df.head()

Let's remove some departments which are not concerned by possible moves.

In [ ]:
deps = df.dep.unique()
deps = deps[deps != '971']
deps = deps[deps != '972']
deps = deps[deps != '973']
deps = deps[deps != '974']
deps = deps[deps != '976']
deps = deps[deps != '978']
deps = deps[~pd.isna(deps)]

Let's create a dictionnary of aggregated data per department.

In [ ]:
hosp_data = {}
for d in deps:
    data = df[df.dep==d]
    data = data.groupby('jour').sum().reset_index().drop(columns=['sexe'])
    data['jour']=pd.to_datetime(data['jour'])
    data = data.sort_values(by=['jour'], ascending=False)
    hosp_data[d] = data    

We will use some GPS data on departments from the uploaded data asset. Refer to the instruction on how to insert the code to read this data. 

In [ ]:
## insert the read departements data asset code here. See instructions. 

Execute the "strip_zero" function.  This is used to fix incompatibilities is using the "deps" list data with the deps_data series data structure.

In [ ]:
def strip_zero(val):
    if val[0]=='0':
        return (str(int(val)))
    return val

Convert the departments LONGITUDE and LATITUDE from strings to real numbers.

In [ ]:
def coord(val):
    neg = (val[-1] == 'O')
    #print (pos)
    c1 = val.split('d')[0]
    #print (c1)
    c2 = val.split('d')[1].split("'")[0]
    #print (c2)
    c3 = val.split('d')[1].split("'")[1].split('"')[0]
    #print (c3)
    res = float(c1) + float(c2)/60 + float(c3)/3600
    if neg:
        res = -res
    return res
    

deps_data['LONGITUDE'] = deps_data['LONGITUDE'].apply(lambda x: coord(x))
deps_data['LATITUDE'] = deps_data['LATITUDE'].apply(lambda x: coord(x))

deps_data = deps_data.set_index(['Nd'])
deps_data.head()

Determine the distance between two locations. 

In [ ]:
def distance(lt1, lg1, lt2, lg2):

    R = 6373.0
    
    lat1 = math.radians(lt1)
    lon1 = math.radians(lg1)
    lat2 = math.radians(lt2)
    lon2 = math.radians(lg2)

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = math.sin(dlat / 2)**2 + math.cos(lat1) * math.cos(lat2) * math.sin(dlon / 2)**2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))

    distance = R * c
    return distance
    

### 2. Display the current situation on a map

We will use folium to represent the data on maps.

In [ ]:
!pip install folium

Let's now represent the situation for each department

In [ ]:
import folium 

m = folium.Map(
    location=[45.5236, 5.6750],
    zoom_start=6
)

initial = {}
capacity = {}

for d, dep in deps_data.iterrows():  
    if (d in ['1','2','3','4','5','6','7','8','9']):
        d = '0' + str(d)

    sz = int(hosp_data[d]['rea'].iat[0])
    initial[d] = sz
    capacity[d] = int(1.2*dep['CAPACITY'])    
    
url = 'https://france-geojson.gregoiredavid.fr/repo/departements.geojson'
def style_function(feature):
    d = feature['properties']['code']
    return {
        'fillOpacity': 0.4 if initial[d]>capacity[d] else 0,
        'weight': 0.5,
        'color': 'black',
        'fillColor': 'red' if initial[d]>capacity[d] else 'white'
    }

folium.GeoJson(
    url,
    name='geojson',
    style_function=style_function
).add_to(m)

for d, dep in deps_data.iterrows():    
    lt = dep['LATITUDE']
    lg = dep['LONGITUDE']
    nm = dep['DEPARTEMENT']
    
    if (d in ['1','2','3','4','5','6','7','8','9']):
        d = '0' + str(d)

    sz = int(hosp_data[d]['rea'].iat[0])
    initial[d] = sz
    #folium.Marker([lt, lg]).add_to(here_map)    
    color = 'green'
    if initial[d]>0.5*capacity[d]:
        color = 'orange'
    if initial[d]>capacity[d]:
        color='red'
    folium.Circle(
        radius=sz*50,
        location=[lt, lg],
        popup=folium.Popup(nm + '(' + str(d)+')<br>'+str(sz)+' reas.<br>Capacity: '+str(capacity[d]),max_width=450,min_width=150),
        color=color,
        fill=True,
        fill_color=color
    ).add_to(m)


m

Create dataframe with status before transfers and calculate the total over capacity

In [ ]:
before = pd.DataFrame(columns=['Nd', 'capacity', 'value', 'overcapacity'], data=[[d, capacity[d], initial[d], max(0, initial[d]-capacity[d])] for d in deps])
print (before)

total_overall_before = before['overcapacity'].sum()
print(total_overall_before)

### 3. Predict the new cases

We use a very dummy linear regression to predict the new cases to appear on each department.

In [ ]:
import numpy as np
from sklearn.linear_model import LinearRegression

   
NB_PERIODS = 3

def predict_more(d, n):   
    X = hosp_data[d].index.tolist()
    X.reverse()
    X=np.array(X).reshape(-1,1)
    y = hosp_data[d]['rea'].tolist()
    y.reverse()
    y=np.array(y).reshape(-1,1)

    regsr=LinearRegression()
    regsr.fit(X,y)

    to_predict_x = [i for i in range(len(X), len(X)+n)]
    to_predict_x= np.array(to_predict_x).reshape(-1,1)
    predicted_y= regsr.predict(to_predict_x)
    delta_y = [ int(round(max(0, predicted_y[i][0]-y[len(y)-1][0]))) for i in range(n)]
    return delta_y

new_rea ={ d:predict_more(d, NB_PERIODS) for d in deps}
print (new_rea)

### 4. Optimize the transfers using Decision Optimization

Now let's create an optimization model.

**Not knowing the real capacities and the real constraints and objectives, we have imagined that some limited short and long transfers could be organized in order to minimize the total over capacity.**

In [ ]:
from docplex.mp.environment import Environment
env = Environment()
env.print_information()

Parameters are the number of possible moves, and the maximum number of cases per move.

In [ ]:
from docplex.mp.model import Model
mdl = Model("Plan Transfers")

MAX_NB_LONG_TRANSFERS_PER_PERIOD = 3
MAX_CASES_PER_LONG_TRANSFERS = 20

MAX_NB_SHORT_TRANSFERS_PER_DEPARTMENT = 3

LONG_DISTANCE = 200

For each "Department" determine if the distance between itself and all other departments is either a "SHORT" or "LONG" transportation route.

In [ ]:
is_long = {d1: {d2: distance(deps_data['LATITUDE'][strip_zero(d1)], deps_data['LONGITUDE'][strip_zero(d1)], deps_data['LATITUDE'][strip_zero(d2)], deps_data['LONGITUDE'][strip_zero(d2)]) > LONG_DISTANCE for d2 in deps} for d1 in deps}

Create a "Plan Transfers" short transfer bounds.

In [ ]:
transfer_periods = list(range(NB_PERIODS))
all_periods = list(range(NB_PERIODS+1))

# Binary vars representing the "assigned" libraries for each coffee shop
use_link_vars = mdl.binary_var_cube(deps, deps, transfer_periods, name="use_link")
link_vars = mdl.integer_var_cube(deps, deps, transfer_periods, lb=0, ub=MAX_CASES_PER_LONG_TRANSFERS, name="link")
occupancy_vars = mdl.integer_var_matrix(deps, all_periods, lb=0, name="occupancy")

# Short transfers bounds
mdl.add_constraints(link_vars[d1, d2, t] <= 1 for d1 in deps for d2 in deps if not is_long[d1][d2] for t in transfer_periods)

mdl.print_information()

Add the contraints 


In [ ]:
# Initial state
mdl.add_constraints(occupancy_vars[d, 0] == initial[d] for d in deps)

# structural constraint between user_link and link
mdl.add_constraints(use_link_vars[d, d1, t] == (link_vars[d, d1, t] >= 1) for d in deps for d1 in deps for t in transfer_periods)

# number of transfers from a department less than current number of cases
mdl.add_constraints(mdl.sum(link_vars[d, d1, t] for d1 in deps) <= occupancy_vars[d, t] for d in deps for t in transfer_periods)

# maximum number of LONG transfers
mdl.add_constraints(mdl.sum(use_link_vars[d1, d2, t] for d1 in deps for d2 in deps if is_long[d1][d2]) <= MAX_NB_LONG_TRANSFERS_PER_PERIOD for t in transfer_periods)


# maximum number of SHORT transfers
mdl.add_constraints(mdl.sum(use_link_vars[d1, d2, t] for d1 in deps if not is_long[d1][d2] for t in transfer_periods) <= MAX_NB_SHORT_TRANSFERS_PER_DEPARTMENT for d2 in deps )

# conservation constraints including new cases to come
mdl.add_constraints(occupancy_vars[d, t+1] == new_rea[d][t] + occupancy_vars[d, t] + mdl.sum(link_vars[d1, d, t] for d1 in deps) - mdl.sum(link_vars[d, d1, t] for d1 in deps) for d in deps for t in transfer_periods)

mdl.print_information()

Objective is to minimize the total over capacity  at the end

In [ ]:
final_overcapacity = mdl.sum(mdl.max(0,  occupancy_vars[d, NB_PERIODS] - capacity[d]) for d in deps)
mdl.add_kpi(final_overcapacity)

nb_long_transfers = mdl.sum(use_link_vars[d1, d2, t] for d1 in deps for d2 in deps if is_long[d1][d2] for t in transfer_periods)
mdl.add_kpi(nb_long_transfers)

nb_short_transfers = mdl.sum(use_link_vars[d1, d2, t] for d1 in deps for d2 in deps if not is_long[d1][d2] for t in transfer_periods)
mdl.add_kpi(nb_short_transfers)

mdl.minimize(1000 * final_overcapacity + 10 * nb_long_transfers + nb_short_transfers)

Have the "docplex" solve the optimization model.

In [ ]:
mdl.set_time_limit(20) 
mdl.solve(log_output=True)

Extract and print the moves

In [ ]:
edges = [(t, d1, d2, int(link_vars[d1, d2, t]), is_long[d1][d2]) for t in transfer_periods for d1 in deps for d2 in deps if int(link_vars[d1, d2, t]) >= 1]
print (edges)

Recreate structure for final situation and calculate final over capacity.

In [ ]:
final = { d: int(occupancy_vars[d, NB_PERIODS].solution_value) for d in deps }

after = pd.DataFrame(columns=['Nd', 'capacity', 'value', 'overcapacity'], data=[[d, capacity[d], final[d], max(0, final[d]-capacity[d])] for d in deps])
print(after)

total_overall_after = after['overcapacity'].sum()
print(total_overall_after)

### 5. Display the solution

And represent the moves and the changes for impacted departments.

* Black links are the local transfers.
* Green links are the long distance transfers.

In [ ]:
def style_function(feature):
    d = feature['properties']['code']
    return {
        'fillOpacity': 0.4 if final[d]>capacity[d] else 0,
        'weight': 0.5,
        'color': 'black',
        'fillColor': 'red' if final[d]>capacity[d] else 'white'
    }


def get_angle(p1, p2):
    
    '''
    This function Returns angle value in degree from the location p1 to location p2
    
    Parameters it accepts : 
    p1 : namedtuple with lat lon
    p2 : namedtuple with lat lon
    
    This function Return the vlaue of degree in the data type float
    
    Pleae also refers to for better understanding : https://gist.github.com/jeromer/2005586
    '''
    
    longitude_diff = np.radians(p2.lon - p1.lon)
    
    latitude1 = np.radians(p1.lat)
    latitude2 = np.radians(p2.lat)
    
    x_vector = np.sin(longitude_diff) * np.cos(latitude2)
    y_vector = (np.cos(latitude1) * np.sin(latitude2) 
        - (np.sin(latitude1) * np.cos(latitude2) 
        * np.cos(longitude_diff)))
    angle = np.degrees(np.arctan2(x_vector, y_vector))
    
    # Checking and adjustring angle value on the scale of 360
    if angle < 0:
        return angle + 360
    return angle


def get_arrows(locations, color, size, n_arrows):
    
    '''
    Get a list of correctly placed and rotated 
    arrows/markers to be plotted
    
    Parameters
    locations : list of lists of lat lons that represent the 
                start and end of the line. 
                eg [[41.1132, -96.1993],[41.3810, -95.8021]]
    arrow_color : default is 'blue'
    size : default is 6
    n_arrows : number of arrows to create.  default is 3
    Return
    list of arrows/markers
    '''
    
    Point = collections.namedtuple('Point', field_names=['lat', 'lon'])
    
    # creating point from our Point named tuple
    p1 = Point(locations[0][0], locations[0][1])
    p2 = Point(locations[1][0], locations[1][1])
    
    # getting the rotation needed for our marker.  
    # Subtracting 90 to account for the marker's orientation
    # of due East(get_bearing returns North)
    rotation = get_angle(p1, p2) - 90
    
    # get an evenly space list of lats and lons for our arrows
    # note that I'm discarding the first and last for aesthetics
    # as I'm using markers to denote the start and end
    arrow_lats = np.linspace(p1.lat, p2.lat, n_arrows + 2)[1:n_arrows+1]
    arrow_lons = np.linspace(p1.lon, p2.lon, n_arrows + 2)[1:n_arrows+1]
    
    arrows = []
    
    #creating each "arrow" and appending them to our arrows list
    for points in zip(arrow_lats, arrow_lons):
        arrows.append(folium.RegularPolygonMarker(location=points, 
                      fill_color=color, number_of_sides=3, 
                      radius=size, rotation=rotation))
    return arrows



m = folium.Map(
    location=[45.5236, 5.6750],
    zoom_start=6
)

    
url = 'https://france-geojson.gregoiredavid.fr/repo/departements.geojson'


folium.GeoJson(
    url,
    name='geojson',
    style_function=style_function
).add_to(m)


for d, dep in deps_data.iterrows():  
    if (d in ['1','2','3','4','5','6','7','8','9']):
        d = '0' + str(d)

    if final[d]==initial[d]:
        continue
    lt = dep['LATITUDE']
    lg = dep['LONGITUDE']
    nm = dep['DEPARTEMENT']
    sz0 = initial[d]
    sz1 = final[d]
    color = 'green'
    if final[d]>0.5*capacity[d]:
        color = 'orange'
    if final[d]>capacity[d]:
        color='red'    
    folium.Circle(
        radius=sz1*50,
        location=[lt, lg],
        popup=folium.Popup(nm+' (' + str(d)+'):<br>From '+str(sz0)+' to '+str(sz1)+' reas.<br>Capacity: '+str(capacity[d]),max_width=450,min_width=150),
        color=color,
        fill=True,
        fill_color=color
    ).add_to(m)
        
for (t, c, b, v, il) in edges:            
    c = strip_zero(c)
    b = strip_zero(b)
    ic = deps_data.index.get_loc(c)
    ib = deps_data.index.get_loc(b)
    dep1 = deps_data.iloc[ic]
    dep2 = deps_data.iloc[ib]
    coordinates = [[dep1['LATITUDE'], dep1['LONGITUDE']], [dep2['LATITUDE'], dep2['LONGITUDE']]]
    color = 'green'
    weight = 4
    if not il:
        color = 'black'
        weight = 2
    pl = folium.PolyLine(coordinates, color=color, weight=weight)
    m.add_child(pl)
    
    arrows = get_arrows(locations=coordinates, color=color, size=3, n_arrows=3)
    for arrow in arrows:
        arrow.add_to(m)
    
    
    
m